# DOUBLE CHECKER FOR IFAS CITATIONS

In [1]:
#@title Run for imports
import requests
from pathlib import Path
import pandas as pd
import numpy as np
import re

In [83]:
fname = '2022 CREC'
path = Path('drive/MyDrive/colab_data/IFAS')
journals = pd.read_excel(path/'journals.xlsx', usecols="A,D", sheet_name="Confirmed")
journals_new = pd.read_excel(path/'journals.xlsx', usecols="A,D", sheet_name="New in 2022")
sheet = pd.read_excel(path/f'{fname}.xlsx', usecols="A:I", sheet_name=fname)

##Make sure Journals are valid

In [84]:
#@title Find Journals (Nothing displays yet)
def find_jour(name):
  found = journals.loc[(journals["Journal Title"].str.lower().replace('[^a-zA-Z0-9]', '', regex=True).str.strip() == re.sub('[^a-zA-Z0-9]', '', name.lower()))].shape
  if found[0] >= 1:
    return True, journals['Notes (confirmed; where; when; initials)']
  found = journals_new.loc[(journals_new["Journal Title"].str.lower().replace('[^a-zA-Z0-9]', '', regex=True).str.strip()  == re.sub('[^a-zA-Z0-9]', '', name.lower()))].shape
  if found[0] >= 1:
    return True, journals['Notes (confirmed; where; when; initials)']
  return False, np.NaN

df = pd.DataFrame(sheet['Journal Title'].copy())
for i,j in enumerate(sheet['Journal Title']):
  found, comm = find_jour(j)
  if found:
    df.at[i, "Found"] = 'Yes'
  else:
    df.at[i, "Found"] = 'No'

In [85]:
#@title Formatting results
pd.set_option("max_colwidth", None)
found = df.loc[df.Found == "Yes"]['Journal Title'].str.lower()
found = found.str.replace(' ', '')
found = found.str.replace('&', '')
found = found.str.replace('and', '')

n_jour = journals.copy()
n_jour['Journal Title'] = n_jour['Journal Title'].str.lower()
n_jour['Journal Title'] = n_jour['Journal Title'].str.replace(' ', '')
n_jour['Journal Title'] = n_jour['Journal Title'].str.replace('&','')
n_jour['Journal Title'] = n_jour['Journal Title'].str.replace('and','')

o_jour = journals_new.copy()
o_jour['Journal Title'] = o_jour['Journal Title'].str.lower()
o_jour['Journal Title'] = o_jour['Journal Title'].str.replace(' ', '')
o_jour['Journal Title'] = o_jour['Journal Title'].str.replace('&', '')
o_jour['Journal Title'] = o_jour['Journal Title'].str.replace('and', '')

Journals that weren't found:

In [86]:
sheet.loc[sheet['Journal Title'] == "Andrology"]

,Author(s),Year,Article Title,Journal Title,Volume,Issue,Pages/Article Number,IFAS Unit,doi


In [87]:
df.loc[df.Found == "No"]

,Journal Title,Found
4,CABI Compendium,No
50,Cells,No
91,LWT,No


Comments of journals that were found (to make sure they are peer reviewed):

In [88]:
journals.loc[(n_jour['Journal Title']).isin(found)]

,Journal Title,Notes (confirmed; where; when; initials)
10,Acta Horticulturae,Confirmed; mea 2/27/2020. Confirmed acceptable with Dr. Adams even though it's symposium 1/30/23- BZ
43,Agriculture,1/28/22 confirmed; http://ulrichsweb.serialssolutions.com/title/1643390957456/714428; BF
44,Agriculture,Confirmed; https://www.mdpi.com/journal/agriculture; mea 1/30/2020
54,Agronomy,Confirmed; https://www.mdpi.com/journal/agronomy; mea 2/6/2019
66,American Journal of Agricultural Economics,Confirmed 2/25/19 CO
122,Antibiotics,Confirmed; https://www.mdpi.com/journal/antibiotics; mea 2/4/2020
145,Applied Soil Ecology,Confrimed 2/26/19 CO
209,Biocontrol,Confirmed; https://www.springer.com/life+sciences/entomology/journal/10526/PSE?detailsPage=press; mea 2/14/2019
232,Biology,Confirmed; https://www.mdpi.com/journal/biology; mea 2/4/2019
275,BMC Plant Biology,Confrimed 2/26/19 CO


In [89]:
journals_new.loc[(o_jour['Journal Title']).isin(found)]

,Journal Title,Notes (confirmed; where; when; initials)
3,Saudi Journal of Biological Sciences,confirmed; http://ulrichsweb.serialssolutions.com/title/1674569058831/704244; 1/24/2023; AG
4,Food Science and Human WellNess,confirmed; http://ulrichsweb.serialssolutions.com/title/1674575679708/755435; 1/24/2023; AG
5,Frontiers in Water,confirmed; http://ulrichsweb.serialssolutions.com/title/1674578504870/844261; 1/24/2023; AG
99,Environmental Entomology,"confirmed; ulrichs, 2/2/23 bz"


##Check page formats are valid

In [90]:
sheet.replace('*', np.nan, inplace=True)

In [91]:
def checkPageFormats():
  x = sheet.loc[((sheet['Pages/Article Number'].str.contains('-') == True) & (sheet['Pages/Article Number'].str[:2] == '1-')) | 
                (sheet['Pages/Article Number'].str.contains('-') == False) & (sheet['Pages/Article Number'].str[0] != '#') | sheet['Pages/Article Number'].isna()]
  if x.shape[0] == 0:
    return "All pages look good, format-wise"
  else:
    return x

checkPageFormats()

,Author(s),Year,Article Title,Journal Title,Volume,Issue,Pages/Article Number,IFAS Unit,doi
2,"Ribeiro, C., Xu, J., Hendrich, C., Pandey, S. S., Yu, Q., Gmitter, F., & Wang, N.",NaN,Seasonal transcriptome profiling of susceptible and tolerant citrus cultivars to citrus huanglongbing,Phytopathology,NaN,NaN,NaN,2022 CREC,10.1094/phyto-05-22-0179-r
4,"Wang, N.",2022.0,Candidatus Liberibacter asiaticus (citrus greening),CABI Compendium,CABI Compendium,NaN,NaN,2022 CREC,10.1079/cabicompendium.16565
14,"Bernardini, C., Turner, D., Wang, C., Welker, S., Achor, D., Artiga, Y. A., Turgeon, R., & Levy, A.",2022.0,Candidatus Liberibacter asiaticus accumulation in the phloem inhibits callose and reactive oxygen species,Plant Physiology,190,2,1090–1094,2022 CREC,https://doi.org/10.1093/plphys/kiac346


In [92]:
n_found = sheet

**Missing Volumes**

In [93]:
n_found.loc[(n_found['Volume'] == "*") | (n_found['Volume'].isna() == True)]

,Author(s),Year,Article Title,Journal Title,Volume,Issue,Pages/Article Number,IFAS Unit,doi
2,"Ribeiro, C., Xu, J., Hendrich, C., Pandey, S. S., Yu, Q., Gmitter, F., & Wang, N.",NaN,Seasonal transcriptome profiling of susceptible and tolerant citrus cultivars to citrus huanglongbing,Phytopathology,NaN,NaN,NaN,2022 CREC,10.1094/phyto-05-22-0179-r


**Missing Issues**

In [94]:
pd.reset_option('colwidth')
pd.set_option('display.max_rows', None)
n_found.loc[(n_found['Issue'] == "*") | (n_found['Issue'].isna() == True)]

,Author(s),Year,Article Title,Journal Title,Volume,Issue,Pages/Article Number,IFAS Unit,doi
2,"Ribeiro, C., Xu, J., Hendrich, C., Pandey, S. ...",NaN,Seasonal transcriptome profiling of susceptibl...,Phytopathology,NaN,NaN,NaN,2022 CREC,10.1094/phyto-05-22-0179-r
4,"Wang, N.",2022.0,Candidatus Liberibacter asiaticus (citrus gree...,CABI Compendium,CABI Compendium,NaN,NaN,2022 CREC,10.1079/cabicompendium.16565
5,"Adhikari, B., & Vashishth, T.",2022.0,‘Just the right amount’ of hydrogen cyanamide ...,Acta Horticulturae,1344,NaN,143-150,2022 CREC,10.17660/ActaHortic.2022.1344.22
16,"Britt-Ugartemendia, K., Turner, D., Sieburth, ...",2022.0,Survey and detection for citrus tristeza virus...,Frontiers in Plant Science,13,NaN,#1050650,2022 CREC,10.3389/fpls.2022.1050650
17,"Canton, M., Wu, H., Dutt, M., & Zale, J.",2022.0,A new liquid selection system for mature citru...,Scientia Horticulturae,293,NaN,#110672,2022 CREC,https://doi.org/10.1016/j.scienta.2021.110672
18,"Castellano-Hinojosa, A., Martens-Habbena, W., ...",2022.0,Short-term effects of cover crops on soil prop...,Applied Soil Ecology,172,NaN,#104341,2022 CREC,https://doi.org/10.1016/j.apsoil.2021.104341
20,"Chen, X. D., Stockton, D., Gossett, H., Quresh...",2022.0,Comparisons of economic thresholds for Asian c...,Frontiers in Sustainable Food Systems,6,NaN,#948278,2022 CREC,10.3389/fsufs.2022.948278
21,"Coetzee, B., Carstens, E., Fourie, P. H., Dewd...",2022.0,Discerning the global phylogeographic distribu...,Fungal Genetics and Biology,162,NaN,#103727,2022 CREC,https://doi.org/10.1016/j.fgb.2022.103727
26,"Dunn, L. L., Friedrich, L. M., Strawn, L. K., ...",2022.0,Prevalence of Listeria monocytogenes and indic...,Food Microbiology,104,NaN,#103970,2022 CREC,https://doi.org/10.1016/j.fm.2021.103970
28,"Dutt, M., Mahmoud, L. M., Nehela, Y., Grosser,...",2022.0,The Citrus sinensis tiller angle control 1 (Cs...,Plant Science,323,NaN,#111401,2022 CREC,10.1016/j.plantsci.2022.111401


##Get Guideline information

In [73]:
#@title Search Function
def search(title, author, first_res=False):
  title.replace(' ', '')
  response = requests.get(f'https://api.crossref.org/works?query.title={title}&query.author={author}&select=author,published,title,volume,issue,article-number,page,DOI,URL,type')
  try:
    for i in range(min(15, len(response.json()['message']['items']))):
      article = response.json()['message']['items'][i]
      if i == 0:
        first_res = article
      
      if article['title'][0].lower() == title.lower():
        return article
      
    if first_res:
      return first_res
  except:
    print(response.text)
  
  return None

In [74]:
#@title Search Function 2
def newSearch(df_rep, title, author, first_res=False):
  data = search(df_rep[title], df_rep[author].split(',')[0])
  try:
    df_rep['F_title'] = data['title'][0]
  except:
    df_rep['F_title'] = np.NaN
  try:
    df_rep['F_page'] = data['page']
  except:
    df_rep['F_page'] = np.NaN
  try:
    df_rep['F_iss'] = data['issue']
  except:
    df_rep['F_iss'] = np.NaN
  try:
    df_rep['F_vol'] = data['volume']
  except:
    df_rep['F_vol'] = np.NaN
  try:
    df_rep['F_type'] = data['type']
  except:
    df_rep['F_type'] = np.NaN
  try:
    df_rep['F_doi'] = data['DOI']
  except:
    df_rep['F_doi'] = np.NaN
  try:
    df_rep['URL'] = data['URL']
  except:
    df_rep['URL'] = np.NaN
  try:
    for a in data['author']:
      if len(a['affiliation']) == 0:
        continue
      for aff in a['affiliation']:
        if 'of florida' in aff['name'].lower():
          df_rep['UF_Affil'] = True
          return df_rep
        else:
          df_rep['UF_Affil'] = False
  except:
    df_rep['UF_Affil'] = np.NaN
  
  return df_rep


In [95]:
new_sheet = sheet

In [96]:
new_sheet = new_sheet.apply(newSearch, axis=1, args=('Article Title', 'Author(s)'))
n_sheet = new_sheet[['Article Title', 'F_title',	'Author(s)', 'Issue',	'F_iss', 'Pages/Article Number',	'F_page', 'Volume', 'F_vol','F_type',	'Journal Title', 'UF_Affil', 'Year','doi', 'F_doi', 'URL']]

In [97]:
pd.reset_option('colwidth')
n_s = n_sheet.copy()
n_s['Issue'] = n_sheet['Issue'].astype(str)
n_s['Issue'] = n_s['Issue'].replace('NaN', np.NaN)
n_s['Article Title'] = n_s['Article Title'].str.lower().replace('[^a-zA-Z0-9]', '', regex=True).str.strip()
n_s['F_title'] = n_s['F_title'].str.lower().replace('[^a-zA-Z0-9]', '', regex=True).str.strip()


n_found = n_sheet.loc[(n_s['Article Title'].str.len() == n_s['F_title'].str.len())]
n_miss = n_sheet.loc[(n_s['Article Title'].str.len() != n_s['F_title'].str.len())]

**Rows that MAY have formatting problems in title:**

In [98]:
pd.set_option("max_colwidth", None)
n_miss.loc[((n_miss['Issue'].astype(str) == n_miss['F_iss'].astype(str))) | ((n_s['Article Title'].str[:10] == n_s['F_title'].str[:10]) & (n_s['F_title'].str.contains('<') == False))]

,Article Title,F_title,Author(s),Issue,F_iss,Pages/Article Number,F_page,Volume,F_vol,F_type,Journal Title,UF_Affil,Year,doi,F_doi,URL
1,‘Candidatus Liberibacter’ pathosystems at the forefront of agricultural and biological research challenges,‘<i>Candidatus</i>Liberibacter’ Pathosystems at the Forefront of Agricultural and Biological Research Challenges,"Pierson, E. A., Cubero, J., Roper, C., Brown, J. K., Bock, C. H., & Wang, N.",1,1,7-10,7-10,112,112,journal-article,Phytopathology,True,2022.0,10.1094/phyto-12-21-0497-fi,10.1094/phyto-12-21-0497-fi,http://dx.doi.org/10.1094/phyto-12-21-0497-fi
14,Candidatus Liberibacter asiaticus accumulation in the phloem inhibits callose and reactive oxygen species,<i>Candidatus</i> Liberibacter asiaticus accumulation in the phloem inhibits callose and reactive oxygen species,"Bernardini, C., Turner, D., Wang, C., Welker, S., Achor, D., Artiga, Y. A., Turgeon, R., & Levy, A.",2,2,1090–1094,1090-1094,190,190,journal-article,Plant Physiology,True,2022.0,https://doi.org/10.1093/plphys/kiac346,10.1093/plphys/kiac346,http://dx.doi.org/10.1093/plphys/kiac346
19,Monitoring of Diaphorina citri populations from Florida reveals reduced susceptibility to cyantraniliprole and thiamethoxam,Monitoring of\n <i>Diaphorina citri</i>\n populations from Florida reveals reduced susceptibility to cyantraniliprole and thiamethoxam,"Chen, X. D., Qureshi, J. A., & Stelinski, L. L.",6,6,725-733,725-733,146,146,journal-article,Journal of Applied Entomology,True,2022.0,https://doi.org/10.1111/jen.13011,10.1111/jen.13011,http://dx.doi.org/10.1111/jen.13011
22,Transcriptome profiling of ‘Candidatus liberibacter asiaticus’ in citrus and psyllids,Transcriptome Profiling of ‘<i>Candidatus</i>Liberibacter asiaticus’ in Citrus and Psyllids,"De Francesco, A., Lovelace, A. H., Shaw, D., Qiu, M., Wang, Y., Gurung, F., Ancona, V., Wang, C., Levy, A., Jiang, T., & Ma, W.",1,1,116-130,116-130,112,112,journal-article,Phytopathology,True,2022.0,10.1094/phyto-08-21-0327-fi,10.1094/phyto-08-21-0327-fi,http://dx.doi.org/10.1094/phyto-08-21-0327-fi
25,Rapid evaluation of the resistance of citrus germplasms against Xanthomonas citri subsp. citri,Rapid Evaluation of the Resistance of Citrus Germplasms Against <i>Xanthomonas citri</i> subsp. <i>citri</i>,"Duan, S., Long, Y., Cheng, S., Li, J., Ouyang, Z., & Wang, N.",4,4,765-774,765-774,112,112,journal-article,Phytopathology,True,2022.0,10.1094/phyto-04-21-0175-r,10.1094/phyto-04-21-0175-r,http://dx.doi.org/10.1094/phyto-04-21-0175-r
31,"Growth, consumption and digestive enzyme activities of Spodoptera littoralis (Boisd) on various mung bean cultivars reveal potential tolerance traits","Growth, consumption and digestive enzyme activities of\n <i>Spodoptera littoralis</i>\n (Boisd) on various mung bean cultivars reveal potential tolerance traits","Fard, S. Z., Hemmati, S. A., Shishehbor, P., & Stelinski, L. L.",9,9,1145-1154,1145-1154,146,146,journal-article,Journal of Applied Entomology,True,2022.0,https://doi.org/10.1111/jen.13055,10.1111/jen.13055,http://dx.doi.org/10.1111/jen.13055
33,"Detectability of hibiscus mealybug, Nipaecoccus viridis (Hemiptera: Pseudoccocidae), DNA in the mealybug destroyer, Cryptolaemus montrouzieri (Coleoptera: Coccinellidae), and survey of its predators in Florida citrus groves","Detectability of Hibiscus Mealybug, <i>Nipaecoccus viridis</i> (Hemiptera: Pseudoccocidae), DNA in the Mealybug Destroyer, <i>Cryptolaemus montrouzieri</i> (Coleoptera: Coccinellidae), and Survey of Its Predators in Florida Citrus Groves","Gaines, K. C., Stelinski, L. L., Neupane, S., & Diepenbrock, L. M.",5,5,1583-1591,1583-1591,115,115,journal-article,Journal of Economic Entomology,True,2022.0,10.1093/jee/toac080,10.1093/jee/toac080,http://dx.doi.org/10.1093/jee/toac080
41,Comparative transcriptomic analysis on compatible/incompatible grafts in Citrus,Comparative transcriptomic analysis on compatible/incompatible grafts in <i>Citrus</i>,"He, W., Xie, R., Wang, Y., Chen, Q., Wang,

**Search Found that there was no UF Affiliation**

In [99]:
n_found.loc[n_found['UF_Affil'] == False]

,Article Title,F_title,Author(s),Issue,F_iss,Pages/Article Number,F_page,Volume,F_vol,F_type,Journal Title,UF_Affil,Year,doi,F_doi,URL


In [ ]:
n_found.loc[n_found['UF_Affil'] == True]

**Article type may not be journal article**

In [81]:
n_found.loc[n_found['F_type'].str[0] != "j"]

,Article Title,F_title,Author(s),Issue,F_iss,Pages/Article Number,F_page,Volume,F_vol,F_type,Journal Title,UF_Affil,Year,doi,F_doi,URL
0,Expansion of information in scientific research papers,Expansion of information in scientific research papers,"Abdullaeva, M., Bromfield, J. J., & Sheldon, I. M.",NaN,NaN,NaN,NaN,NaN,NaN,posted-content,bioRxiv,NaN,2022.0,10.1101/2022.05.06.490896,10.1101/2022.05.06.490896,http://dx.doi.org/10.1101/2022.05.06.490896


Original Issue was Missing, but search found an issue number

In [82]:
n_found.loc[(n_found['Issue'].isna() == True) & (n_found['F_iss'].isna() == False)]

,Article Title,F_title,Author(s),Issue,F_iss,Pages/Article Number,F_page,Volume,F_vol,F_type,Journal Title,UF_Affil,Year,doi,F_doi,URL
30,Social networks respond to a disease challenge in calves,Social networks respond to a disease challenge in calves,"Burke, K. C., do Nascimento-Emond, S., Hixson, C. L., & Miller-Cushon, E. K.",NaN,1,#9119,NaN,12.0,12,journal-article,Scientific Reports,NaN,2022.0,10.1038/s41598-022-13088-2,10.1038/s41598-022-13088-2,http://dx.doi.org/10.1038/s41598-022-13088-2
72,Dietary selenium across species,Dietary Selenium Across Species,"Lei, X. G., Combs, G. F., Sunde, R. A., Caton, J. S., Arthington, J. D., & Vatamaniuk, M. Z.",NaN,1,337-375,337-375,42.0,42,journal-article,Annual Review of Nutrition,True,2022.0,10.1146/annurev-nutr-062320-121834,10.1146/annurev-nutr-062320-121834,http://dx.doi.org/10.1146/annurev-nutr-062320-121834
79,Socioecological predictors of breastfeeding practices in rural eastern Ethiopia,Socioecological predictors of breastfeeding practices in rural eastern Ethiopia,"Magalhães, M., Ojeda, A., Mechlowitz, K., Brittain, K., Daniel, J., Roba, K. T., Hassen, J. Y., Manary, M. J., Gebreyes, W. A., Havelaar, A. H., & McKune, S. L.",NaN,1,#93,NaN,17.0,17,journal-article,International Breastfeeding Journal,NaN,2022.0,10.1186/s13006-022-00531-3,10.1186/s13006-022-00531-3,http://dx.doi.org/10.1186/s13006-022-00531-3
88,Megasphaera elsdenii and Saccharomyces cerevisiae as direct fed microbials during an in vitro acute ruminal acidosis challenge,Megasphaera elsdenii and Saccharomyces Cerevisiae as direct fed microbials during an in vitro acute ruminal acidosis challenge,"Monteiro, H. F., Agustinho, B. C., Vinyard, J. R., Harden, T., Bennett, S. L., Arce-Cordero, J. A., Sarmikasoglou, E., Ravelo, A. D., Bahman, A., So, S., Vieira, E. R., & Faciola, A. P.",NaN,1,#7978,NaN,12.0,12,journal-article,Scientific Reports,NaN,2022.0,10.1038/s41598-022-11959-2,10.1038/s41598-022-11959-2,http://dx.doi.org/10.1038/s41598-022-11959-2
